In [ ]:
import numpy as np
import sympy as sp

# Theory

Last time we discussed linear systems of ODEs like

\begin{align*}
x' &= x + 2y \\
y' &= 2x + y
\end{align*}

We wrote the system as a matrix equation:

$$
\frac{\text{d}\mathbf{x}}{\text{d}t} = \mathbf{A}\mathbf{x}
$$

where 

\begin{align*}
&\mathbf{x}(t) = \begin{pmatrix}
             x(t) \\
             y(t) \end{pmatrix} \\
\\
&\mathbf{A} = \begin{pmatrix}
             1 & 2 \\
             2 & 1 \end{pmatrix}
\end{align*}

and used SymPy to compute the **Matrix Exponential** of $\mathbf{A}t$ to write down the solution 

$$
\mathbf{x}(t) = e^{\mathbf{A}t}\mathbf{x}(0)
$$

So the equation, in matrix form, reads:

$$
\frac{\text{d}}{\text{d}t} \begin{pmatrix}
             x \\
             y \end{pmatrix} = \begin{pmatrix}
             1 & 2 \\
             2 & 1 \end{pmatrix} \begin{pmatrix}
             x \\
             y \end{pmatrix} = \begin{pmatrix} x + 2y \\ 2x + y \end{pmatrix}
$$





----

The matrix exponential of a square matrix $\mathbf{A}$ is another square matrix of the same shape definined by the convergent infinite series 

\begin{align*}
e^{\mathbf{A}} &:= \sum_{n=0}^\infty \frac{\mathbf{A}^n}{n!} \\
               & = \mathbf{I} + \mathbf{A} + \frac{\mathbf{A}^2}{2} + \frac{\mathbf{A}^3}{6} + \dots
\end{align*}

For **diagonalisable** matrices $\mathbf{A}$ the exponential $e^\mathbf{A}$ can be computed by leveraging the diagonalisation. 

A **diagonalisation** of a diagonalisable matrix $\mathbf{A}$ is a factorisation of the form

$$
\mathbf{A} = \mathbf{P} \mathbf{D} \mathbf{P}^{-1}
$$

where $\mathbf{P}$ and $\mathbf{D}$ are two square matrices of the same shape as $\mathbf{A}$. The matrix $\mathbf{P}$ is built by making the eigenvectors of $\mathbf{A}$ the columns of a square matrix, while $\mathbf{D}$ is a diagonal matrix whose diagonal entries are precisely the eigenvalues of $\mathbf{A}$

The matrix exponential is then convenientally given by (why?)

$$
e^{\mathbf{A}} = \mathbf{P} e^{\mathbf{D}} \mathbf{P}^{-1}
$$

where $e^\mathbf{D}$ is easily computed, because the matrix exponential of a diagonal matrix is the new diagonal matrix obtained by exponentiating the diagonal entries of $\mathbf{D}$.


---

For example, if 

$$
\mathbf{D} = \begin{pmatrix} 1 & 0 \\ 0 & 2 \end{pmatrix}
$$

then computing the matrix exponential is really easy, it is just

$$
e^\mathbf{D} = \begin{pmatrix} e^1 & 0 \\ 0 & e^2 \end{pmatrix}
$$


----

In this notebook we'll walk through computing the matrix exponential $e^{\mathbf{A}t}$ for the matrix 

$$
\mathbf{A} = \begin{pmatrix}
             1 & 2 \\
             2 & 1 \end{pmatrix}
$$

we used last time.

We'll compare the results we get by directly using `sympy.exp()` on the matrix vs. computing the matrix exponential by using the diagonalisation

If 

$$
\mathbf{A} = \begin{pmatrix}
             1 & 2 \\
             2 & 1 \end{pmatrix}
$$

Then by $\mathbf{A}t$ I just mean

$$
\mathbf{A}t = \begin{pmatrix}
             t & 2t \\
             2t & t \end{pmatrix}
$$

This product $\mathbf{A}t$ is relevant because it appears in the general solution of a linear system of ODEs:

$$
\mathbf{x}(t) = e^{\mathbf{A}t}\mathbf{x}(0)
$$

# Code

## Matrix Exponential with `sympy.exp`

In [47]:
A = sp.Matrix([[1, 2], 
               [2, 1]])
A

Matrix([
[1, 2],
[2, 1]])

Last time we called `sympy.exp` as a black box and didn't even define the matrix exponential until the end of the session

In [48]:
# Computing e^At by using the inbuilt SymPy matrix exponential

t = sp.symbols('t')
exp_At = sp.exp(A * t)

exp_At


Matrix([
[exp(3*t)/2 + exp(-t)/2, exp(3*t)/2 - exp(-t)/2],
[exp(3*t)/2 - exp(-t)/2, exp(3*t)/2 + exp(-t)/2]])

## Diagonalising $\mathbf{A}$

### Eigenvalues and the matrix $\mathbf{D}$

To diagonalise the matrix $\mathbf{A}$ we need to construct the matrices $\mathbf{P}$ and $\mathbf{D}$. In order to do this, we need to find the eigenvalues and eigenvectors of $\mathbf{A}$.

`A.eigenvals()` returns a dictionary, where the key : value pairs are the eigenvalues and the number of times they're repeated (*multiplicities*)

In [18]:
# Computing eigenvalues of A

A.eigenvals()

{-2: 1, 3: 1, -1: 1}

In [53]:
# Turning the dictionary into a list:

eigenvalues = [val for val, mult in A.eigenvals().items() for _ in range(mult)]
eigenvalues

[3, -1]

`sympy.diag` turns a list into a diagonal matrix, where the lists' entries become the diagonal entries of a diagonal matrix

In [54]:
# Constructing the diagonal matrix D used in the diagonalisation of A = P * D * P^-1

D = sp.diag(*eigenvalues)
D

Matrix([
[3,  0],
[0, -1]])

### Eigenvectors and the matrix $\mathbf{P}$

The output of `A.eigenvects()` is a list of tuples, where each tuple has the following structure:

1. **Eigenvalue**: A scalar value $\lambda$.
2. **Algebraic Multiplicity**: An integer indicating the multiplicity of the eigenvalue (number of times it is repeated)
3. **Eigenvectors**: A list of eigenvectors (each a column vector as a `Matrix` object) corresponding to the eigenvalue.

Example:
[ 
($\lambda_1$, $m_1$, [$v_{11}$, $v_{12}$, $\dots$]), 
($\lambda_2$, $m_2$, [$v_{21}$, $\dots$]), 
$\dots$ 
]


To extract the first eigenvector for the first eigenvalue:
```python
eigenvectors[0][2][0]


In [55]:
# Computing the eigenvectors of A

eigenvectors = A.eigenvects()
eigenvectors


[(-1,
  1,
  [Matrix([
   [-1],
   [ 1]])]),
 (3,
  1,
  [Matrix([
   [1],
   [1]])])]

In [58]:
eigenvectors[0]

(-1,
 1,
 [Matrix([
  [-1],
  [ 1]])])

In [61]:
eigenvectors[0][2]

[Matrix([
 [-1],
 [ 1]])]

In [62]:
# So the λ = -1 eigenvector is 
eigenvectors[0][2][0]

Matrix([
[-1],
[ 1]])

In [63]:
# And the λ = 3 eigenvector is
eigenvectors[1][2][0]

Matrix([
[1],
[1]])

Thus the eigenvectors are 

$$
\begin{pmatrix}
    1 \\ 1
\end{pmatrix}
$$

corresponding to the eigenvalue $\lambda = 3$ and 

$$
\begin{pmatrix}
    -1 \\ 1
\end{pmatrix}
$$

corresponding to the eigenvalue $\lambda = -1$.

The matrix $\mathbf{P}$ is constructed by concatenating the eigenvectors into a square matrix. (In this order, to match the order of the eigenvalues in the matrix $\mathbf{D}$). 

That is, 

$$ \mathbf{P} = \begin{pmatrix} 1 & -1 \\ 1 & 1 \end{pmatrix} $$

In SymPy we can achieve this with `sympy.Matrix.hstack`


In [64]:
# Constructing the matrix P used in the diagonalisation A = P * D * P^-1 by concatenating the eigenvectors into a matrix

P = sp.Matrix.hstack(eigenvectors[1][2][0], eigenvectors[0][2][0]) # The eigenvectors need to go into P in the same order that the eigenvalues went into D
P


Matrix([
[1, -1],
[1,  1]])

### Completing the diagonalisation

The diagonalisation of $\mathbf{A}$ is the factorisation of matrices

$$
\mathbf{A} = \mathbf{P} \mathbf{D} \mathbf{P}^{-1}
$$

In our case, this is 

$$
\begin{pmatrix}
1 & 2 \\
2 & 1
\end{pmatrix}
=
\begin{pmatrix}
1 & -1 \\
1 & 1
\end{pmatrix}
\begin{pmatrix}
3 & 0 \\
0 & -1
\end{pmatrix}
\begin{pmatrix}
1 & -1 \\
1 & 1
\end{pmatrix}^{-1}
$$

We can verify this indeed is true before proceeding to calculate the matrix exponential


In [65]:
# Define P^-1 (the inverse of P)
P_inv = P.inv()
P_inv


Matrix([
[ 1/2, 1/2],
[-1/2, 1/2]])

In [66]:
# Computing PDP^-1 to see if this does actually return A

A_reconstructed = P * D * P_inv
A_reconstructed

Matrix([
[1, 2],
[2, 1]])

In [67]:
A == A_reconstructed

True

### Using the diagonalisation to compute the matrix exponential

In [68]:
# Multiply the matrices A and D by a variable 't' in order to calculate e^At

t = sp.symbols('t')

At = A*t
Dt = D*t

At

Matrix([
[  t, 2*t],
[2*t,   t]])

In [69]:
Dt

Matrix([
[3*t,  0],
[  0, -t]])

In [70]:
# Verifying that our diagonalisation still holds 

At_reconstructed = P * Dt * P_inv
At_reconstructed

Matrix([
[  t, 2*t],
[2*t,   t]])

In [71]:
# The matrix exponential of a diagonal matrix is just the new diagonal matrix formed by exponentiating each of the diagonal entries

exp_Dt = sp.diag(*[sp.exp(val) for val in Dt.diagonal()])
exp_Dt

Matrix([
[exp(3*t),       0],
[       0, exp(-t)]])

In [72]:
# Computing the matrix exponential twice:
# 1) by lazily calling `sp.exp` as before
# 2) by leveraging the diagonalisation we have built:   At   = P *   Dt *   P^-1
#                                                ==>  exp_At = P * exp_Dt * P^-1

exp_At_viaSymPy = sp.exp(A * t)
exp_At_viaDiagonalisation = P * exp_Dt * P_inv

exp_At_viaDiagonalisation

Matrix([
[exp(3*t)/2 + exp(-t)/2, exp(3*t)/2 - exp(-t)/2],
[exp(3*t)/2 - exp(-t)/2, exp(3*t)/2 + exp(-t)/2]])

In [73]:
exp_At_viaDiagonalisation == exp_At_viaSymPy

True

# Exercises

## 1)

Consider the matrix

$$
\mathbf{A} = 
\begin{pmatrix}
-1 & -19 & -4 \\
0 & -2 & 0 \\
0 & 15 & 3
\end{pmatrix}
$$


- i) Find the eigenvalues and eigenvectors of $\mathbf{A}$ using SymPy
- ii) Use the eigenvalues and eigenvectors to contruct the matrices $\mathbf{P}$ and $\mathbf{D}$
- iii) Compute $\mathbf{PDP}^{-1}$
- iv) Compute the matrix exponential $e^\mathbf{A}$ by using the diagonalisation $A = \mathbf{PDP}^{-1}$

In [6]:
import sympy as sp

A_1 = sp.Matrix([[-1, -19, -4],
[0, -2, 0],
[0, 15, 3]])

A_1

Matrix([
[-1, -19, -4],
[ 0,  -2,  0],
[ 0,  15,  3]])

In [ ]:
A_1_EigVec = A_1.eigenvects()

A_1_EigVec



[(-2,
  1,
  [Matrix([
   [-7/3],
   [-1/3],
   [   1]])]),
 (-1,
  1,
  [Matrix([
   [1],
   [0],
   [0]])]),
 (3,
  1,
  [Matrix([
   [-1],
   [ 0],
   [ 1]])])]

In [ ]:
#find P

first_eig = A_1_EigVec[0][2][0]
second_eig = A_1_EigVec[1][2][0]
third_eig = A_1_EigVec[2][2][0]

P_1 = sp.Matrix.hstack(first_eig, second_eig, third_eig)

P_1


Matrix([
[-7/3, 1, -1],
[-1/3, 0,  0],
[   1, 0,  1]])

In [ ]:
# Find inverse of P
P_inv_1 = P_1.inv()

P_inv_1

Matrix([
[0, -3, 0],
[1, -4, 1],
[0,  3, 1]])

In [ ]:
# Find D

first_eigval = A_1_EigVec[0][0]
second_eigval = A_1_EigVec[1][0]
third_eigval = A_1_EigVec[2][0]


D_1 = sp.diag(first_eigval, second_eigval, third_eigval)
D_1


Matrix([
[-2,  0, 0],
[ 0, -1, 0],
[ 0,  0, 3]])

In [34]:
answer_1 = P_1 * D_1 * P_inv_1
answer_1

Matrix([
[-1, -19, -4],
[ 0,  -2,  0],
[ 0,  15,  3]])

In [35]:
answer_1 == A_1

True

In [44]:
#find e^eigenvalues


exp_D_1 = sp.diag(sp.exp(first_eigval), sp.exp(second_eigval), sp.exp(third_eigval))


# calculate matrix exp of A

A_1_exp = P_1 * exp_D_1 * P_inv_1
A_1_exp

Matrix([
[exp(-1), -3*exp(3) - 4*exp(-1) + 7*exp(-2), -exp(3) + exp(-1)],
[      0,                           exp(-2),                 0],
[      0,             -3*exp(-2) + 3*exp(3),            exp(3)]])

In [45]:
## Compare with the result we get by directly calling `sp.exp` on A

A_1_exp_lazy = sp.exp(A)
A_1_exp_lazy

Matrix([
[exp(-1), -3*exp(3) - 4*exp(-1) + 7*exp(-2), -exp(3) + exp(-1)],
[      0,                           exp(-2),                 0],
[      0,             -3*exp(-2) + 3*exp(3),            exp(3)]])

## 2)

Consider the matrix

$$
\mathbf{A} = 
\begin{pmatrix}
4 & -9 & 6 & 12 \\
0 & -1 & 4 & 6 \\
2 & -11 & 8 & 16 \\
-1 & 3 & 0 & -1
\end{pmatrix}
$$


- i) Find the eigenvalues and eigenvectors of $\mathbf{A}$ using SymPy
- ii) Use the eigenvalues and eigenvectors to contruct the matrices $\mathbf{P}$ and $\mathbf{D}$
- iii) Compute $\mathbf{PDP}^{-1}$
- iv) Compute the matrix exponential $e^\mathbf{A}$ by using the diagonalisation $A = \mathbf{PDP}^{-1}$

---
# Theory

Last time we discussed linear systems of ODEs like

\begin{align*}
x' &= x + 2y \\
y' &= 2x + y
\end{align*}

We wrote the system as a matrix equation:

$$
\frac{\text{d}\mathbf{x}}{\text{d}t} = \mathbf{A}\mathbf{x}
$$

where 

\begin{align*}
&\mathbf{x}(t) = \begin{pmatrix}
             x(t) \\
             y(t) \\ z(t) \end{pmatrix} \\
\\

\end{align*}

and used SymPy to compute the **Matrix Exponential** of $\mathbf{A}t$ to write down the solution 

$$
\mathbf{x}(t) = e^{\mathbf{A}t}\mathbf{x}(0)
$$





---

## 3

Consider the linear system of ODEs

\begin{align*}
x' &= x + 4z \\
y' &= 2y \\
z' &= 3x + y - 3z
\end{align*}

with initial conditions $x(0) = 0$, $y(0) = 0$ and $z(0) = 4$

- i) Define the matrix $A$ for the system
- ii) Compute the matrix exponential $e^{\mathbf{A}t}$ by calling `sympy.exp`
- iii) Compute the matrix exponential $e^{\mathbf{A}t}$ by finding a diagonalisation of $\mathbf{A}$ and compare your result with the result of part ii)
- iv) Use the matrix exponential to find the solution of the system of ODEs

In [48]:
A_3 = sp.Matrix([[1,0,4],
[0,2,0],
[3,1,-3]])

A_3

Matrix([
[1, 0,  4],
[0, 2,  0],
[3, 1, -3]])

In [49]:
A_3_exp_easy = sp.exp(A_3)